In [ ]:
!pip install statsmodels

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import glob
import os
import fnmatch
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels.stats.api as sms
import colorsys
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)



## Baseline based on the normal noise model
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180225_035402_mini_imagenet_repeat_num_self_attention_splits_task_encoder_self_attention/'
experiement_patterns = ['*num_self_attention_splits=1*', '*num_self_attention_splits=2*', 
                        '*num_self_attention_splits=4*']

# ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180225_122144_mini_imagenet_repeat_num_self_attention_splits_fc_dropout_task_encoder_self_attention/'
# experiement_patterns = ['*fc_dropout=0.2;num_self_attention_splits=4*',
#                        '*fc_dropout=None;num_self_attention_splits=4*',
#                        '*fc_dropout=0.5;num_self_attention_splits=4*',
#                        '*fc_dropout=0.2;num_self_attention_splits=1*',
#                        '*fc_dropout=None;num_self_attention_splits=1*',
#                        '*fc_dropout=0.5;num_self_attention_splits=1*']
# experiement_patterns = ['*fc_dropout=None*num_self_attention_splits=4*',
#                        '*fc_dropout=None*num_self_attention_splits=1*']

ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180227_034716_mini_imagenet_task_encoder_repeat_attention_num_filters_embedding_pooled_task_encoder_self_attention_patch/'
experiement_patterns = ['*task_encoder=label_embed*attention_num_filters=64;embedding_pooled=True*',
                        '*task_encoder=self_attention*attention_num_filters=64;embedding_pooled=True*',
                        '*task_encoder=label_embed*attention_num_filters=32;embedding_pooled=True*',
                        '*task_encoder=self_attention*attention_num_filters=32;embedding_pooled=True*',
                        '*task_encoder=label_embed*attention_num_filters=16;embedding_pooled=True*',
                        '*task_encoder=self_attention*attention_num_filters=16;embedding_pooled=True*',]

# experiement_patterns = ['*attention_num_filters=64;embedding_pooled=True*',
#                         '*attention_num_filters=32;embedding_pooled=True*',
#                         '*attention_num_filters=16;embedding_pooled=True*',]

ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180227_213958_mini_imagenet_repeat_attention_num_filters_class_embed_size_task_encoder_new_hyperparams_test/'
experiement_patterns = ['*attention_num_filters=64*class_embed_size=16*',
                        '*attention_num_filters=64*class_embed_size=32*',
                        '*attention_num_filters=64*class_embed_size=64*',
                        '*attention_num_filters=128*class_embed_size=16*',
                        '*attention_num_filters=128*class_embed_size=32*',
                        '*attention_num_filters=128*class_embed_size=64*',]

ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180228_121015_mini_imagenet_repeat_attention_num_filters_class_embed_size_task_encoder_new_hyperparams_test/'
experiement_patterns = ['*attention_num_filters=64*class_embed_size=16*',
                        '*attention_num_filters=64*class_embed_size=32*',
                        '*attention_num_filters=64*class_embed_size=64*',
                        '*attention_num_filters=32*class_embed_size=16*',
                        '*attention_num_filters=32*class_embed_size=32*',
                        '*attention_num_filters=32*class_embed_size=64*',]


# ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180301_011738_mini_imagenet_repeat_number_of_steps_task_encoder_num_steps/'
# experiement_patterns = ['*number_of_steps=104000*']


ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180301_112415_mini_imagenet_repeat_weights_initializer_factor_task_encoder_num_steps/'
experiement_patterns = ['*weights_initializer_factor=1*',
                        '*weights_initializer_factor=0.5*',
                        '*weights_initializer_factor=0.2*',]


ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180301_112900_mini_imagenet_num_attention_models_repeat_task_encoder_num_attention_models/'
experiement_patterns = ['*num_attention_models=2*',
                        '*num_attention_models=4*',
                        '*num_attention_models=6*',]

# 6 attention models
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180302_015239_mini_imagenet_repeat_weights_initializer_factor_task_encoder_6attention_models_noise_weight/'
experiement_patterns = ['*weights_initializer_factor=0.5*',
                        '*weights_initializer_factor=0.2*',
                        '*weights_initializer_factor=0.1*',]

# 6 attention models
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180302_113000_mini_imagenet_repeat_weights_initializer_factor_task_encoder_6attention_models_noise_weight/'
experiement_patterns = ['*weights_initializer_factor=0.5*',
                        '*weights_initializer_factor=0.2*',
                        '*weights_initializer_factor=0.1*',]

# Test highway and new implementation of sum 
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180302_194337_mini_imagenet_repeat_attention_fusion_task_encoder_testing_highway/'
experiement_patterns = ['*attention_fusion=sum*','*attention_fusion=highway*',]

# Test batch size 16
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180304_170453_mini_imagenet_repeat_number_of_steps_task_encoder_testing_highway/'
experiement_patterns = ['*number_of_steps=156000*','*number_of_steps=208000*',]

# FC dropout experiement
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180305_011527_mini_imagenet_repeat_number_of_steps_fc_dropout_task_encoder_fc_dropout/'
experiement_patterns = ['*number_of_steps=156000*fc_dropout=None*',
                       '*number_of_steps=156000*fc_dropout=0.1*',
                       '*number_of_steps=156000*fc_dropout=0.2*',
                       '*number_of_steps=156000*fc_dropout=0.5*',
                       '*number_of_steps=208000*fc_dropout=None*',
                       '*number_of_steps=208000*fc_dropout=0.1*',
                       '*number_of_steps=208000*fc_dropout=0.2*',
                       '*number_of_steps=208000*fc_dropout=0.5*',
                       ]

# Cosine learning rate annealing experiment
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180305_035219_mini_imagenet_repeat_task_encoder_cos_anneal/'
experiement_patterns = ['*repeat*', ]

# Experiment with more than one task per batch. This is usng piecewise constant anneal schedule 1/2, 3/4, 7/8
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180305_183801_mini_imagenet_repeat_num_tasks_per_batch_train_batch_size_task_encoder_cos_anneal'
experiement_patterns = ['*train_batch_size=32*num_tasks_per_batch=1*',
                       '*train_batch_size=32*num_tasks_per_batch=2*',
                       '*train_batch_size=32*num_tasks_per_batch=3*',
                       '*train_batch_size=16*num_tasks_per_batch=2*',
                       '*train_batch_size=16*num_tasks_per_batch=3*',]

# # Experiment with more than one task per batch. This is usng exponential equidistant learning rate decay with 3 segments
# ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180305_211049_mini_imagenet_repeat_num_tasks_per_batch_train_batch_size_task_encoder_cos_anneal/'
# experiement_patterns = ['*train_batch_size=32*num_tasks_per_batch=1*', 
#                         '*train_batch_size=16*num_tasks_per_batch=2*', 
#                         '*train_batch_size=32*num_tasks_per_batch=2*',
#                         '*train_batch_size=8*num_tasks_per_batch=3*', 
#                         '*train_batch_size=16*num_tasks_per_batch=3*',
#                         '*train_batch_size=32*num_tasks_per_batch=3*',
#                        ]

# # Baseline with 2 tasks per batch
# ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180306_122149_mini_imagenet_repeat_number_of_steps_train_batch_size_lr_anneal_task_encoder_2tasks_batch/'
# experiement_patterns = ['*train_batch_size=3*number_of_steps=1*lr_anneal=pwc*']

# Experiment with 2, 3, 4 tasks per batch
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180306_184001_mini_imagenet_repeat_num_tasks_per_batch_task_encoder_234tasks_batch_pwc/'
experiement_patterns = ['*num_tasks_per_batch=2*', '*num_tasks_per_batch=3*', '*num_tasks_per_batch=4*']


ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180307_061837_mini_imagenet_number_of_steps_feat_extract_pretrain_repeat_task_encoder_pretrain_test/'
experiement_patterns = ['*number_of_steps=5*feat_extract_pretrain=multitask*',
                        '*number_of_steps=5*feat_extract_pretrain=finetune*',
                        '*number_of_steps=5*feat_extract_pretrain=None*',
                        '*number_of_steps=1*feat_extract_pretrain=multitask*',
                        '*number_of_steps=1*feat_extract_pretrain=finetune*',
                        '*number_of_steps=1*feat_extract_pretrain=None*'
                       ]


ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180307_184608_mini_imagenet_number_of_steps_feat_extract_pretrain_feat_extract_pretrain_lr_decay_n_repeat_task_encoder_pretrain_exploration/'
experiement_patterns = ['*number_of_steps=5*feat_extract_pretrain_lr_decay_n=3*feat_extract_pretrain=multitask*',
                        '*number_of_steps=5*feat_extract_pretrain_lr_decay_n=3*feat_extract_pretrain=finetune*',
                        '*number_of_steps=5*feat_extract_pretrain_lr_decay_n=2*feat_extract_pretrain=multitask*',
                        '*number_of_steps=5*feat_extract_pretrain_lr_decay_n=2*feat_extract_pretrain=finetune*',
                        '*number_of_steps=2*feat_extract_pretrain_lr_decay_n=3*feat_extract_pretrain=multitask*',
                        '*number_of_steps=2*feat_extract_pretrain_lr_decay_n=3*feat_extract_pretrain=finetune*',
                        '*number_of_steps=2*feat_extract_pretrain_lr_decay_n=2*feat_extract_pretrain=multitask*',
                        '*number_of_steps=2*feat_extract_pretrain_lr_decay_n=2*feat_extract_pretrain=finetune*',
                       ]

# Experiment with auxiliary task selection probability
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180308_163335_mini_imagenet_number_of_steps_feat_extract_pretrain_decay_rate_repeat_num_classes_pretrain_task_encoder_multitask_disjoint_datasets/'
experiement_patterns = ['*number_of_steps=26000*feat_extract_pretrain_decay_rate=0.8*',
                        '*number_of_steps=26000*feat_extract_pretrain_decay_rate=0.9;*',
                        '*number_of_steps=26000*feat_extract_pretrain_decay_rate=0.99*',
                        '*number_of_steps=52000*feat_extract_pretrain_decay_rate=0.8*',
                        '*number_of_steps=52000*feat_extract_pretrain_decay_rate=0.9;*',
                        '*number_of_steps=52000*feat_extract_pretrain_decay_rate=0.99*',]

# New baseline, 8 attention splits
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180309_034509_mini_imagenet_num_attention_models_task_encoder_num_attention_layers_repeat_task_encoder_pretrain_test/'
experiement_patterns = ['*num_attention_models=8*num_attention_layers=1*',
                       '*num_attention_models=6*num_attention_layers=1*',
                       '*num_attention_models=1*num_attention_layers=1*',]


# New baseline, 8, 12, 16 attention splits and 32, 64, 128 filters
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180309_124026_mini_imagenet_num_attention_models_attention_num_filters_repeat_task_encoder_attention_filters_splits/'
experiement_patterns = ['*num_attention_models=16*attention_num_filters=128*',
                       '*num_attention_models=16*attention_num_filters=64*',
                       '*num_attention_models=16*attention_num_filters=32*',]

# Experiment, small number of steps
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180309_183505_mini_imagenet_number_of_steps_class_embed_size_repeat_task_encoder_class_embedding/'
experiement_patterns = ['*number_of_steps=15600*class_embed_size=32*',
                       '*number_of_steps=15600*class_embed_size=64*',]


# # Experiment, not sharing the class embeddinggs
# ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180309_205949_mini_imagenet_num_attention_layers_repeat_task_encoder_sharing_task_encoder_task_embedding_sharing/'
# experiement_patterns = ['*num_attention_layers=1*task_encoder_sharing=None*',
#                        '*num_attention_layers=1*task_encoder_sharing=layer*',
#                        '*num_attention_layers=1*task_encoder_sharing=global*',]


# Experiment, not sharing the class embeddinggs
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180310_151540_mini_imagenet_number_of_steps_class_embed_size_repeat_task_encoder_sharing_task_encoder_embedding_size_8_16_32/'
experiement_patterns = ['*number_of_steps=26000*class_embed_size=16;task_encoder_sharing=None*',
                       '*number_of_steps=26000*class_embed_size=16;task_encoder_sharing=layer*',
                       '*number_of_steps=26000*class_embed_size=32;task_encoder_sharing=None*',
                       '*number_of_steps=26000*class_embed_size=32;task_encoder_sharing=layer*',
                       '*number_of_steps=15600*class_embed_size=16;task_encoder_sharing=None*',
                       '*number_of_steps=15600*class_embed_size=16;task_encoder_sharing=layer*',
                       '*number_of_steps=15600*class_embed_size=32;task_encoder_sharing=None*',
                       '*number_of_steps=15600*class_embed_size=32;task_encoder_sharing=layer*',
                       '*number_of_steps=26000*class_embed_size=8;task_encoder_sharing=None*',
                       '*number_of_steps=26000*class_embed_size=8;task_encoder_sharing=layer*',
                       '*number_of_steps=15600*class_embed_size=8;task_encoder_sharing=None*',
                       '*number_of_steps=15600*class_embed_size=8;task_encoder_sharing=layer*',
                       ]

# Experiment, not sharing the class embeddinggs, more runs
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180311_041110_mini_imagenet_number_of_steps_class_embed_size_repeat_task_encoder_sharing_task_encoder_embedding_size_8_16_32/'
experiement_patterns = ['*number_of_steps=26000*class_embed_size=8;task_encoder_sharing=None*',
                       '*number_of_steps=26000*class_embed_size=8;task_encoder_sharing=layer*',
                       '*number_of_steps=26000*class_embed_size=32;task_encoder_sharing=None*',
                       '*number_of_steps=26000*class_embed_size=32;task_encoder_sharing=layer*',
                       '*number_of_steps=15600*class_embed_size=8;task_encoder_sharing=None*',
                       '*number_of_steps=15600*class_embed_size=8;task_encoder_sharing=layer*',
                       '*number_of_steps=15600*class_embed_size=32;task_encoder_sharing=None*',
                       '*number_of_steps=15600*class_embed_size=32;task_encoder_sharing=layer*',
                       ]

# Ablation study prototypical head, resnet, sgd, multitask
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180313_001359_mini_imagenet_encoder_classifier_link_feature_extractor_repeat_embedding_pooled_task_encoder_feature_extractor_ablation_more_steps/'
experiement_patterns = ['*encoder_classifier_link=prototypical*feature_extractor=simple_conv_net;embedding_pooled=True*',
                       '*encoder_classifier_link=prototypical*feature_extractor=simple_conv_net;embedding_pooled=False*',
                        '*encoder_classifier_link=prototypical*feature_extractor=simple_res_net;embedding_pooled=True*',
                       '*encoder_classifier_link=prototypical*feature_extractor=simple_res_net;embedding_pooled=False*',
                       ]

# Ablation study prototypical head and adam
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180313_112548_mini_imagenet_encoder_classifier_link_feature_extractor_repeat_embedding_pooled_task_encoder_prototypical_adam_no_multitask/'
experiement_patterns = ['*encoder_classifier_link=prototypical*feature_extractor=simple_conv_net;embedding_pooled=True*',
                       '*encoder_classifier_link=prototypical*feature_extractor=simple_conv_net;embedding_pooled=False*',
                        '*encoder_classifier_link=prototypical*feature_extractor=simple_res_net;embedding_pooled=True*',
                       '*encoder_classifier_link=prototypical*feature_extractor=simple_res_net;embedding_pooled=False*',
                       ]

# Ablation study prototypical head and adam, no L2 norm
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180313_164709_mini_imagenet_number_of_steps_feature_extractor_repeat_weight_decay_task_encoder_prototypical/'
experiement_patterns = ['*number_of_steps=20000*weight_decay=0;feature_extractor=simple_conv_net*',
                        '*number_of_steps=20000*weight_decay=0.0005;feature_extractor=simple_conv_net*',
#                         '*number_of_steps=20000*weight_decay=0;feature_extractor=simple_res_net*',
#                         '*number_of_steps=20000*weight_decay=0.0005;feature_extractor=simple_res_net*',
                       ]

# Replicating the prototypical nets results
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180314_014829_mini_imagenet_number_of_steps_repeat_train_batch_size_num_classes_train_task_encoder_prototypical/'
experiement_patterns = ['*train_batch_size=300;number_of_steps=20000*num_classes_train=5*',
                        '*train_batch_size=300;number_of_steps=20000*num_classes_train=20*',
                        '*train_batch_size=300;number_of_steps=40000*num_classes_train=5*',
                        '*train_batch_size=300;number_of_steps=40000*num_classes_train=20*',
                        '*train_batch_size=75;number_of_steps=20000*num_classes_train=5*',
                        '*train_batch_size=75;number_of_steps=20000*num_classes_train=20*',
                        '*train_batch_size=75;number_of_steps=40000*num_classes_train=5*',
                        '*train_batch_size=75;number_of_steps=40000*num_classes_train=20*',
                       ]

# Prototypical nets with res_net feature extractor
# Replicating the prototypical nets results
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180314_021127_mini_imagenet_number_of_steps_feature_extractor_repeat_train_batch_size_num_classes_train_task_encoder_prototypical_20way_resnet/'
experiement_patterns = ['*train_batch_size=300;number_of_steps=20000*feature_extractor=simple_res_net*num_classes_train=5*',
#                         '*train_batch_size=300;number_of_steps=20000*feature_extractor=simple_res_net*num_classes_train=20*',
                        '*train_batch_size=300;number_of_steps=40000*feature_extractor=simple_res_net*num_classes_train=5*',
#                         '*train_batch_size=300;number_of_steps=40000*feature_extractor=simple_res_net*num_classes_train=20*',
                        '*train_batch_size=75;number_of_steps=20000*feature_extractor=simple_res_net*num_classes_train=5*',
                        '*train_batch_size=75;number_of_steps=20000*feature_extractor=simple_res_net*num_classes_train=20*',
                        '*train_batch_size=75;number_of_steps=40000*feature_extractor=simple_res_net*num_classes_train=5*',
                        '*train_batch_size=75;number_of_steps=40000*feature_extractor=simple_res_net*num_classes_train=20*',
                       ]


# Study of multitask training and adam for conv_net, 
# multitask does not help, probably with adam final learning rate for the aux task is not optimal
# No significant difference, so class embedding is all we need
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180315_040436_mini_imagenet_feat_extract_pretrain_repeat_num_classes_train_task_encoder_prototypical_multitask/'
experiement_patterns = [
    '*num_classes_train=5*feat_extract_pretrain=multitask*',
    '*num_classes_train=20*feat_extract_pretrain=multitask*',
    '*num_classes_train=5*feat_extract_pretrain=None*',
    '*num_classes_train=20*feat_extract_pretrain=None*',
]

# Experiemnt excluding the image features from the FC layer in attention model. Only use class embedding
ROOT_DIR = '/mnt/home/boris/experiments_task_encoder/180314_233040_mini_imagenet_num_attention_models_attention_no_original_embedding_task_encoder_sharing_repeat_feat_extract_pretrain_decay_rate_num_classes_train_task_encoder_cut_original_embedding/'
experiement_patterns = [
    "*num_attention_models=16*num_classes_train=5;task_encoder_sharing=None;attention_no_original_embedding=True;feat_extract_pretrain_decay_rate=0.9*",
    "*num_attention_models=16*num_classes_train=5;task_encoder_sharing=None;attention_no_original_embedding=False;feat_extract_pretrain_decay_rate=0.9*",
]

# Experiemnt multi-way training of resnet with adam and euclidian distance
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180314_191612_mini_imagenet_repeat_train_batch_size_num_classes_train_task_encoder_prototypical_multyway_resnet/'
experiement_patterns = [
    "*train_batch_size=75*num_classes_train=5*",
    "*train_batch_size=75*num_classes_train=10*",
    "*train_batch_size=75*num_classes_train=20*",
    "*train_batch_size=75*num_classes_train=30*",
]

# Experiement using normalization by standard deviation in the prototypical context
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180315_153440_mini_imagenet_encoder_classifier_link_repeat_num_classes_train_task_encoder_prototypical_std_norm/'
experiement_patterns = [
    "*encoder_classifier_link=prototypical*num_classes_train=20*",
    "*encoder_classifier_link=prototypical*num_classes_train=5*",
    "*encoder_classifier_link=std_normalized_euc_head*num_classes_train=20*",
    "*encoder_classifier_link=std_normalized_euc_head*num_classes_train=5*",
]

# Experiment with resnet, testing multitask and number of classes while training
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180315_160112_mini_imagenet_num_tasks_per_batch_feat_extract_pretrain_feature_extractor_repeat_num_classes_train_task_encoder_proto_resnet_multitask/'
experiement_patterns = [
    "*num_classes_train=5*num_tasks_per_batch=2;feature_extractor=simple_res_net;feat_extract_pretrain=multitask*",
    "*num_classes_train=20*num_tasks_per_batch=2;feature_extractor=simple_res_net;feat_extract_pretrain=multitask*",
    "*num_classes_train=5*num_tasks_per_batch=2;feature_extractor=simple_res_net;feat_extract_pretrain=None*",
    "*num_classes_train=20*num_tasks_per_batch=2;feature_extractor=simple_res_net;feat_extract_pretrain=None*",
    
]

# Experiemnt multiway training resnet with sgd
'/mnt/home/boris/experiments_task_encoder/180315_205348_mini_imagenet_feat_extract_pretrain_feat_extract_pretrain_lr_decay_n_repeat_num_classes_train_task_encoder_proto_resnet_multitask'

# Experiement Euclidian vs. Cosine distance with adam and non-pooled features
'/mnt/home/boris/experiments_task_encoder/180316_135152_mini_imagenet_encoder_classifier_link_number_of_steps_repeat_num_classes_train_task_encoder_self_attention_euclidian/'

# Experiemnt multiplying cosine distance by a scalar, alexnet and adam
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180317_175217_mini_imagenet_encoder_classifier_link_number_of_steps_repeat_cosine_multiplier_trainable_num_classes_train_cosine_multiplier_init_task_encoder_cosine_scaled_15/'
experiement_patterns = [
    "*num_classes_train=20;encoder_classifier_link=prototypical*number_of_steps=80000*",
    "*num_classes_train=20;encoder_classifier_link=cosine;cosine_multiplier_trainable=False;number_of_steps=80000;cosine_multiplier_init=15",
    "*num_classes_train=5;encoder_classifier_link=prototypical*number_of_steps=80000*",
    "*num_classes_train=5;encoder_classifier_link=cosine;cosine_multiplier_trainable=False;number_of_steps=80000;cosine_multiplier_init=15",
    "*num_classes_train=20;encoder_classifier_link=prototypical*number_of_steps=40000*",
    "*num_classes_train=20;encoder_classifier_link=cosine;cosine_multiplier_trainable=False;number_of_steps=40000;cosine_multiplier_init=15",
    "*num_classes_train=5;encoder_classifier_link=prototypical*number_of_steps=40000*",
    "*num_classes_train=5;encoder_classifier_link=cosine;cosine_multiplier_trainable=False;number_of_steps=40000;cosine_multiplier_init=15",
]

# Experiemnt doing polynomial of euclidian distance, alexnet and adam
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180318_044135_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_num_classes_train_task_encoder_polynomial_metric/'
experiement_patterns = [
    "*encoder_classifier_link=polynomial;number_of_steps=40000*num_classes_train=5;polynomial_metric_order=1",
    "*encoder_classifier_link=polynomial;number_of_steps=80000*num_classes_train=5;polynomial_metric_order=1",
    "*encoder_classifier_link=polynomial;number_of_steps=40000*num_classes_train=5;polynomial_metric_order=5",
    "*encoder_classifier_link=polynomial;number_of_steps=80000*num_classes_train=5;polynomial_metric_order=5",
]

# Experiement estimating a polynomial of the Euclidian distance, resnet and SGD
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180318_054924_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_num_classes_train_task_encoder_polynomial_resnet'
experiement_patterns = [
    "*encoder_classifier_link=prototypical;number_of_steps=26000*num_classes_train=5*",
    "*encoder_classifier_link=prototypical;number_of_steps=26000*num_classes_train=20*",
    "*encoder_classifier_link=prototypical;number_of_steps=52000*num_classes_train=5*",
    "*encoder_classifier_link=prototypical;number_of_steps=52000*num_classes_train=20*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=5*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=5*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=5;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=20;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=5;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=20;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=5;polynomial_metric_order=5*",
#     "*encoder_classifier_link=polynomial;number_of_steps=52000*num_classes_train=20;polynomial_metric_order=5*",
]

# # Experiement estimating a polynomial of the Euclidian distance, resnet and SGD, varying polynomial degrees
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180319_020058_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_num_classes_train_task_encoder_polynomial_perceptron_resnet/'
# experiement_patterns = [
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=3*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=4*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=5;polynomial_metric_order=5*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=1*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=2*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=3*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=4*",
#     "*encoder_classifier_link=polynomial;number_of_steps=26000*num_classes_train=20;polynomial_metric_order=5*",
# ]

# Experiement estimating a polynomial of the Euclidian distance, resnet and SGD, varying polynomial degrees, more repeats
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180319_173649_mini_imagenet_number_of_steps_polynomial_metric_order_repeat_num_classes_train_task_encoder_polynomial_perceptron_resnet/'
experiement_patterns = [
    "*number_of_steps=25000*num_classes_train=5;polynomial_metric_order=1*",
    "*number_of_steps=25000*num_classes_train=5;polynomial_metric_order=2*",
    "*number_of_steps=25000*num_classes_train=5;polynomial_metric_order=3*",
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=3*",
    "*number_of_steps=25000*num_classes_train=10;polynomial_metric_order=1*",
    "*number_of_steps=25000*num_classes_train=10;polynomial_metric_order=2*",
    "*number_of_steps=25000*num_classes_train=10;polynomial_metric_order=3*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=3*",
    "*number_of_steps=25000*num_classes_train=20;polynomial_metric_order=1*",
    "*number_of_steps=25000*num_classes_train=20;polynomial_metric_order=2*",
    "*number_of_steps=25000*num_classes_train=20;polynomial_metric_order=3*",
    "*number_of_steps=50000*num_classes_train=20;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=20;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=20;polynomial_metric_order=3*",
    
]


# Experiement estimating a polynomial of the Euclidian distance, resnet and SGD, varying polynomial degrees, 
# replicating results with tighter range of number_of_steps and num_classes_train, but less repeats
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180320_161741_mini_imagenet_number_of_steps_polynomial_metric_order_repeat_num_classes_train_task_encoder_polynomial_perceptron_resnet/'
experiement_patterns = [
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=3*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=3*",
    "*number_of_steps=50000*num_classes_train=15;polynomial_metric_order=1*",
    "*number_of_steps=50000*num_classes_train=15;polynomial_metric_order=2*",
    "*number_of_steps=50000*num_classes_train=15;polynomial_metric_order=3*",
    "*number_of_steps=40000*num_classes_train=5;polynomial_metric_order=1*",
    "*number_of_steps=40000*num_classes_train=5;polynomial_metric_order=2*",
    "*number_of_steps=40000*num_classes_train=5;polynomial_metric_order=3*",
    "*number_of_steps=40000*num_classes_train=10;polynomial_metric_order=1*",
    "*number_of_steps=40000*num_classes_train=10;polynomial_metric_order=2*",
    "*number_of_steps=40000*num_classes_train=10;polynomial_metric_order=3*",
    "*number_of_steps=40000*num_classes_train=15;polynomial_metric_order=1*",
    "*number_of_steps=40000*num_classes_train=15;polynomial_metric_order=2*",
    "*number_of_steps=40000*num_classes_train=15;polynomial_metric_order=3*",
    "*number_of_steps=60000*num_classes_train=5;polynomial_metric_order=1*",
    "*number_of_steps=60000*num_classes_train=5;polynomial_metric_order=2*",
    "*number_of_steps=60000*num_classes_train=5;polynomial_metric_order=3*",
    "*number_of_steps=60000*num_classes_train=10;polynomial_metric_order=1*",
    "*number_of_steps=60000*num_classes_train=10;polynomial_metric_order=2*",
    "*number_of_steps=60000*num_classes_train=10;polynomial_metric_order=3*",
    "*number_of_steps=60000*num_classes_train=15;polynomial_metric_order=1*",
    "*number_of_steps=60000*num_classes_train=15;polynomial_metric_order=2*",
    "*number_of_steps=60000*num_classes_train=15;polynomial_metric_order=3*",
    
]

# Baseline with a polynomial of the Euclidian distance, resnet and SGD
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180321_150820_mini_imagenet_number_of_steps_polynomial_metric_order_repeat_num_classes_train_polynomial_baseline/'
experiement_patterns = [
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=1*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=1*',
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=2*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=2*',
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=3*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=3*',
]

# Baseline with a polynomial of the Euclidian distance, resnet and SGD, less distance between learning rate annealing events
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180322_002424_mini_imagenet_number_of_steps_polynomial_metric_order_repeat_num_classes_train_polynomial_baseline/'
experiement_patterns = [
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=1*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=1*',
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=2*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=2*',
    '*number_of_steps=50000*num_classes_train=10;polynomial_metric_order=3*',
    '*number_of_steps=50000*num_classes_train=5;polynomial_metric_order=3*',
]

# swish-1, selu, relu experiemnt
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180322_210010_mini_imagenet_number_of_steps_lr_anneal_repeat_activation_num_classes_train_swish-1-test/'
experiement_patterns = [
    '*activation=relu;lr_anneal=pwc;num_classes_train=5*',
    '*activation=selu;lr_anneal=pwc;num_classes_train=5*',
    '*activation=swish-1;lr_anneal=pwc;num_classes_train=5*',
]

# Baseline for the adam based optimizer with simple_conv_net, mini-imagenet, swish-1 activation
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180323_223118_mini_imagenet_encoder_classifier_link_number_of_steps_data_dir_metric_multiplier_init_metric_multiplier_trainable_repeat_num_classes_train_simple_conv_net_mini_imagenet_baseline/'
experiement_patterns = [
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=20000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=20000*',
]

# Baseline for the adam based optimizer with simple_conv_net, mini-imagenet, relu activation
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180326_012431_mini_imagenet_encoder_classifier_link_number_of_steps_data_dir_metric_multiplier_init_metric_multiplier_trainable_repeat_num_classes_train_simple_conv_net_mini_imagenet_baseline_with_relu/'
experiement_patterns = [
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=40000;metric_multiplier_init=15',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=20000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=20000*',
]

# Baseline CIFAR100, simple conv net, prototypical vs cosine
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180326_170423_mini_imagenet_encoder_classifier_link_number_of_steps_metric_multiplier_init_metric_multiplier_trainable_repeat_num_classes_train_simple_conv_net_cifar100/'
experiement_patterns = [
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=5;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=1',
    '*metric_multiplier_trainable=False*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=15',
    '*metric_multiplier_trainable=True*num_classes_train=20;encoder_classifier_link=cosine;number_of_steps=20000;metric_multiplier_init=15',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=40000*',
    '*num_classes_train=5;encoder_classifier_link=prototypical;number_of_steps=20000*',
    '*num_classes_train=20;encoder_classifier_link=prototypical;number_of_steps=20000*',
]

# Experiemnt with resnet feature extractor, mini-Imagenet, shot in train mode different than in the test mode
# New baseline for the polynomial head
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180327_005425_mini_imagenet_encoder_classifier_link_number_of_steps_num_shots_train_repeat_num_classes_train_polynomial_resnet_num_shots_train_steps100000/'
experiement_patterns = [
    '*num_shots_train=5*num_classes_train=5*',
    '*num_shots_train=10*num_classes_train=5*',
]
# Same but with 50000 steps: /mnt/home/boris/experiments_task_encoder/180326_193006_mini_imagenet_encoder_classifier_link_number_of_steps_num_shots_train_repeat_num_classes_train_polynomial_resnet_num_shots_train/

# CBN first successful experiment
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180329_040057_mini_imagenet_number_of_steps_feat_extract_pretrain_feat_extract_pretrain_lr_decay_n_repeat_weight_decay_cbn_cbn_test_MoreMore/'
experiement_patterns = [
    '*weight_decay_cbn=0.01*number_of_steps=3*feat_extract_pretrain_lr_decay_n=2*',
    '*weight_decay_cbn=0.01*number_of_steps=3*feat_extract_pretrain_lr_decay_n=3*',
    '*weight_decay_cbn=0;*number_of_steps=3*feat_extract_pretrain_lr_decay_n=2*',
    '*weight_decay_cbn=0;*number_of_steps=3*feat_extract_pretrain_lr_decay_n=3*',
    '*weight_decay_cbn=0.0005*number_of_steps=3*feat_extract_pretrain_lr_decay_n=2*',
    '*weight_decay_cbn=0.0005*number_of_steps=3*feat_extract_pretrain_lr_decay_n=3*',
]

# CBN vs prototypical with multitask
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180329_204205_mini_imagenet_encoder_classifier_link_number_of_steps_feat_extract_pretrain_repeat_weight_decay_cbn_cbn_vs_prototypical_multitask/'
experiement_patterns = [
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.01*number_of_steps=30000*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.01*number_of_steps=20000*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.0005*number_of_steps=30000*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.0005*number_of_steps=20000*',
    '*encoder_classifier_link=prototypical*number_of_steps=30000*',
    '*encoder_classifier_link=prototypical*number_of_steps=20000*',
]

# CBN with polynomial head vs polynomial with multitask
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180330_022135_mini_imagenet_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_trainable_repeat_weight_decay_cbn_cbn_vs_polynomial_multitask/'
experiement_patterns = [
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.01*metric_multiplier_trainable=False*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.0005*metric_multiplier_trainable=False*',
    '*encoder_classifier_link=polynomial*metric_multiplier_trainable=False*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.01*metric_multiplier_trainable=True*',
    '*encoder_classifier_link=cbn*weight_decay_cbn=0.0005*metric_multiplier_trainable=True*',
    '*encoder_classifier_link=polynomial*metric_multiplier_trainable=True*',
]

# # CBN with separate paramter encoder, with and without multitask multitask
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180330_132040_mini_imagenet_encoder_classifier_link_number_of_steps_feat_extract_pretrain_repeat_weight_decay_cbn_feat_extract_pretrain_lr_decay_rate_separate_task_encoder_cbn/'
# experiement_patterns = [
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=10;encoder_classifier_link=cbn;weight_decay_cbn=0.01;number_of_steps=30000*',
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=100;encoder_classifier_link=cbn;weight_decay_cbn=0.01;number_of_steps=30000*',
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=5;encoder_classifier_link=cbn;weight_decay_cbn=0.01;number_of_steps=30000*',
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=10;encoder_classifier_link=cbn;weight_decay_cbn=0.1;number_of_steps=30000*',
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=100;encoder_classifier_link=cbn;weight_decay_cbn=0.1;number_of_steps=30000*',
#     '*feat_extract_pretrain=multitask;feat_extract_pretrain_lr_decay_rate=5;encoder_classifier_link=cbn;weight_decay_cbn=0.1;number_of_steps=30000*',
# ]

# # CBN with shared parameter encoder, optimizing the multitask parameters
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180331_112834_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_feat_extract_pretrain_decay_rate_weight_decay_cbn_feat_extract_pretrain_lr_decay_rate_shared_task_encoder_cbn_decoupling_layer/'
# experiement_patterns = [
#     '*pretrain_lr_decay_rate=10;*weight_decay_cbn=0.01*pretrain_decay_rate=0.9',
#     '*pretrain_lr_decay_rate=10;*weight_decay_cbn=0.01*pretrain_decay_rate=0.95',
# ]

# # CBN with shared parameter encoder, optimizing the multitask parameters, synchronized lr annealing schedule
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180331_174053_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_feat_extract_pretrain_decay_rate_weight_decay_cbn_multitask_syncronized_lr_decay/'
# experiement_patterns = [
#     '*weight_decay_cbn=0.002;number_of_steps=30000;feat_extract_pretrain_decay_rate=0.95',
#     '*weight_decay_cbn=0.002;number_of_steps=30000;feat_extract_pretrain_decay_rate=0.9',
#     '*weight_decay_cbn=0.01;number_of_steps=30000;feat_extract_pretrain_decay_rate=0.95',
#     '*weight_decay_cbn=0.01;number_of_steps=30000;feat_extract_pretrain_decay_rate=0.9',
# ]

# CBN baseline, 20 retrials
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180401_145432_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_repeat_weight_decay_cbn_cbn_baseline/'
experiement_patterns = [
    '*number_of_steps=30000*',
    '*number_of_steps=20000*',
    '*number_of_steps=40000*',
]

# Multilayer CBN parameter generation and more aggressive rate decay
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180403_110651_mini_imagenet_encoder_classifier_link_number_of_steps_polynomial_metric_order_cbn_num_layers_repeat_weight_decay_cbn_cbn_more_aggressive_lr_decay/'
experiement_patterns = [
    '*cbn_num_layers=1*number_of_steps=30000*',
    '*cbn_num_layers=2*number_of_steps=30000*',
    '*cbn_num_layers=3*number_of_steps=30000*',
]

# # Dataset augmentation
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180411_025808_mini_imagenet_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_num_units_in_block_augment_number_of_steps_cbn_num_layers_repeat_augmentation_test_larger_interval/'
# experiement_patterns = [
#     '*augment=True;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=cbn;number_of_steps=60000*',
#     '*augment=True;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=cbn;number_of_steps=60000*',
#     '*augment=True;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=polynomial;number_of_steps=60000*',
#     '*augment=True;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=polynomial;number_of_steps=60000*',
#     '*augment=True;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=prototypical;number_of_steps=60000*',
#     '*augment=True;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=prototypical;number_of_steps=60000*',
    
#     '*augment=False;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=cbn;number_of_steps=30000*',
#     '*augment=False;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=cbn;number_of_steps=30000*',
#     '*augment=False;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=polynomial;number_of_steps=30000*',
#     '*augment=False;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=polynomial;number_of_steps=30000*',
#     '*augment=False;polynomial_metric_order=1*pretrain=multitask;encoder_classifier_link=prototypical;number_of_steps=30000*',
#     '*augment=False;polynomial_metric_order=1*pretrain=None;encoder_classifier_link=prototypical;number_of_steps=30000*',
# ]

# # CBN num layers and number of CBN per block, MIN
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180412_150135_mini_imagenet_encoder_classifier_link_polynomial_metric_order_feature_extractor_number_of_steps_cbn_per_block_cbn_num_layers_repeat_residense_net_test'
# experiement_patterns = [
#     '*cbn_num_layers=1*cbn_per_block=False;feature_extractor=simple_res_net*',
#     '*cbn_num_layers=1*cbn_per_block=True;feature_extractor=simple_res_net*',
#     '*cbn_num_layers=2*cbn_per_block=False;feature_extractor=simple_res_net*',
#     '*cbn_num_layers=2*cbn_per_block=True;feature_extractor=simple_res_net*',
# ]

# # CBN num layers and polynomial metric order, MIN
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180413_023455_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_polynomial_metric_order_number_of_steps_cbn_per_block_cbn_num_layers_repeat_cbn_layers_poly_order'
# experiement_patterns = [
#     '*cbn_num_layers=1*polynomial_metric_order=1*cbn_per_block=True*',
#     '*cbn_num_layers=1*polynomial_metric_order=2*cbn_per_block=True*',
#     '*cbn_num_layers=1*polynomial_metric_order=3*cbn_per_block=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=1*cbn_per_block=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=2*cbn_per_block=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=3*cbn_per_block=True*',
    
#     '*cbn_num_layers=1*polynomial_metric_order=1*cbn_per_block=False*',
#     '*cbn_num_layers=1*polynomial_metric_order=2*cbn_per_block=False*',
#     '*cbn_num_layers=1*polynomial_metric_order=3*cbn_per_block=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=1*cbn_per_block=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=2*cbn_per_block=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=3*cbn_per_block=False*',
# ]

# # CBN num layer and one CBN per network vs one CBN per max pool
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180420_022402_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_metric_multiplier_init_cbn_per_network_cbn_per_block_cbn_num_layers_repeat_cbn_per_network_test_MIN'
# experiement_patterns = [
#     '*cbn_num_layers=1*polynomial_metric_order=1;cbn_per_network=True*',
#     '*cbn_num_layers=1*polynomial_metric_order=2;cbn_per_network=True*',
#     '*cbn_num_layers=1*polynomial_metric_order=3;cbn_per_network=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=1;cbn_per_network=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=2;cbn_per_network=True*',
#     '*cbn_num_layers=2*polynomial_metric_order=3;cbn_per_network=True*',
#     '*cbn_num_layers=3*polynomial_metric_order=1;cbn_per_network=True*',
#     '*cbn_num_layers=3*polynomial_metric_order=2;cbn_per_network=True*',
#     '*cbn_num_layers=3*polynomial_metric_order=3;cbn_per_network=True*',
    
#     '*cbn_num_layers=1*polynomial_metric_order=1;cbn_per_network=False*',
#     '*cbn_num_layers=1*polynomial_metric_order=2;cbn_per_network=False*',
#     '*cbn_num_layers=1*polynomial_metric_order=3;cbn_per_network=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=1;cbn_per_network=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=2;cbn_per_network=False*',
#     '*cbn_num_layers=2*polynomial_metric_order=3;cbn_per_network=False*',
#     '*cbn_num_layers=3*polynomial_metric_order=1;cbn_per_network=False*',
#     '*cbn_num_layers=3*polynomial_metric_order=2;cbn_per_network=False*',
#     '*cbn_num_layers=3*polynomial_metric_order=3;cbn_per_network=False*',
# ]

# 3,4,5 cbn layers, CBN per block vs CBN for every conv
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180420_191842_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_metric_multiplier_init_cbn_per_block_cbn_num_layers_repeat_more_cbn_layers'
experiement_patterns = [
    '*cbn_num_layers=1*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=False*',
    '*cbn_num_layers=2*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=False*',
    '*cbn_num_layers=3*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=False*',
    '*cbn_num_layers=4*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=False*',
    '*cbn_num_layers=5*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=False*',
    
    '*cbn_num_layers=1*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=True*',
    '*cbn_num_layers=2*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=True*',
    '*cbn_num_layers=3*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=True*',
    '*cbn_num_layers=4*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=True*',
    '*cbn_num_layers=5*polynomial_metric_order=3*feat_extract_pretrain=multitask*cbn_per_block=True*',
]

ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180502_103022_mini_imagenet_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_feat_extract_pretrain_decay_rate_cbn_num_layers_repeat_scale_10_prev_version_test'
experiement_patterns = [
    '*feat_extract_pretrain_decay_rate=0.9;*',
    '*feat_extract_pretrain_decay_rate=0.95*',
]


ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180508_015505_mini_imagenet_num_tasks_per_batch_num_max_pools_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_cbn_num_layers_repeat_maxpool'
experiement_patterns = [
    '*num_tasks_per_batch=2*max_pools=3*metric_multiplier_init=10*',
    '*num_tasks_per_batch=2*max_pools=3*metric_multiplier_init=1*',
]

# Prototypical networks 5-shot and 10-shot with adam
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180512_180858_mini_imagenet_encoder_classifier_link_number_of_steps_metric_multiplier_trainable_repeat_num_classes_train_prototypical_10shot'
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180512_190032_mini_imagenet_encoder_classifier_link_number_of_steps_metric_multiplier_trainable_repeat_num_classes_train_prototypical_5shot'
experiement_patterns = [
    '*number_of_steps=20000*num_classes_train=5*',
    '*number_of_steps=20000*num_classes_train=10*',
    '*number_of_steps=20000*num_classes_train=15*',
    '*number_of_steps=20000*num_classes_train=20*',
]

In [ ]:
import tensorflow as tf; print(tf.__version__)

# Get directories for the  experiements 

In [ ]:
def get_experiment_folder_list(experiement_dir, experiment_pattern):
    list_experiemnts_pattern = []
    list_experiments_all = os.listdir(experiement_dir)
    for folder in list_experiments_all:
        names = os.path.join(ROOT_DIR,folder)
#         print(names)
        names_filt_alone = fnmatch.filter([names], experiment_pattern)
#         print(names_filt_alone)
        list_experiemnts_pattern.extend(names_filt_alone)
    return list_experiemnts_pattern


def interpolate_data(experiment_data, index_max, step):
    index_max = int(round(index_max/step)*step)
    index_new = np.arange(0, index_max+step, step)
    experiment_data_interpolated = {}
    for file, df in experiment_data.items():
        interpolator = interpolate.interp1d(df.index, df.value, bounds_error=False, fill_value=np.nan)
        value_new = interpolator(index_new)
        experiment_data_interpolated[file] = value_new
    
    return pd.DataFrame(index=index_new, data=experiment_data_interpolated)
    

def read_experiment_data(experiement_dir, experiment_pattern):
    experiemnt_folder_list = get_experiment_folder_list(experiement_dir, experiment_pattern)
    experiment_data_tst = {}
    experiment_data_val = {}
    index_max_tst = 0
    index_max_val = 0
    
    i=0    
    for experiement in experiemnt_folder_list:
        print(experiement)
        print(glob.glob1(experiement+'/eval/', 'events.out.tfevents.*'))
        df_tst=[]
        df_val=[]
        for file in glob.glob1(experiement+'/eval/', 'events.out.tfevents.*'):
            ea = EventAccumulator(experiement+'/eval/'+file)
            ea.Reload()
            tags = ea.Tags()['scalars']
#             print(tags)
            if not 'accuracy_target_tst' in tags:
                continue
            data = ea.Scalars('accuracy_target_tst')
            if len(data) > 0:
                df_tst.append(pd.DataFrame(data).set_index('step').drop('wall_time', axis=1))
            data = ea.Scalars('accuracy_target_val')
            if len(data) > 0:
                df_val.append(pd.DataFrame(data).set_index('step').drop('wall_time', axis=1))
        
        if len(df_tst) > 0:
            df_tst = pd.concat(df_tst)

        if len(df_val) > 0:
            df_val = pd.concat(df_val)
        
        if len(df_tst) > 0:
            experiment_data_tst[experiement+file] = df_tst
            index_max_tst = max(index_max_tst, df_tst.index.max())
        
        if len(df_tst) > 0:
            experiment_data_val[experiement+file] = df_val
            index_max_val = max(index_max_val, df_val.index.max())
        
    return interpolate_data(experiment_data_tst, index_max_tst, step=250), interpolate_data(experiment_data_val, index_max_val, step=250)

In [ ]:
list_experiemnts_target_transfer = get_experiment_folder_list(ROOT_DIR, experiement_patterns[0])
list_experiemnts_target_transfer

In [ ]:
plot_data = {}
for experiment_pattern in experiement_patterns:
    print('Loading experiment: %s' %experiment_pattern)
    acc_tst, acc_val = read_experiment_data(ROOT_DIR, experiment_pattern)
    
    plot_data[(experiment_pattern, 'acc_tst')] = acc_tst
    plot_data[(experiment_pattern, 'acc_val')] = acc_val

In [ ]:
# This is to detect runs that did not converge
VALID_RUN_THRESHOLD=0.0

for experiment_pattern in experiement_patterns:
    print('Loading experiment: %s' %experiment_pattern)
    
    acc_tst = plot_data[(experiment_pattern, 'acc_tst')]
    acc_val = plot_data[(experiment_pattern, 'acc_val')]
    
    valid_cols = acc_val.max().index[acc_val.max() > VALID_RUN_THRESHOLD]
    acc_val = acc_val[valid_cols]
    acc_tst = acc_tst[valid_cols]
    
    acc_tst_ci = sms.DescrStatsW(acc_tst.T).tconfint_mean()
    acc_val_ci = sms.DescrStatsW(acc_val.T).tconfint_mean()
    acc_val_mean = acc_val.mean(axis=1)
    acc_tst_mean = acc_tst.mean(axis=1)
    
    plot_data[(experiment_pattern, 'acc_tst_mean')] = acc_tst_mean
    plot_data[(experiment_pattern, 'acc_val_mean')] = acc_val_mean
    plot_data[(experiment_pattern, 'acc_tst_ci')] = acc_tst_ci
    plot_data[(experiment_pattern, 'acc_val_ci')] = acc_val_ci
    plot_data[(experiment_pattern, 'acc_tst')] = acc_tst
    plot_data[(experiment_pattern, 'acc_val')] = acc_val

In [ ]:
colors=[colorsys.hsv_to_rgb(x*1.0/len(experiement_patterns), 1.0, 1.0) for x in range(len(experiement_patterns))]

majorLocator = MultipleLocator(1.0)
majorFormatter = FormatStrFormatter('%.1f')


fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)
for i, experiment_pattern in enumerate(experiement_patterns):
    

    ax1.plot(100.0*plot_data[(experiment_pattern, 'acc_tst_mean')], label=experiment_pattern, c=colors[i])
    ax1.fill_between(plot_data[(experiment_pattern, 'acc_tst_mean')].index, 
                    100.0*plot_data[(experiment_pattern, 'acc_tst_ci')][0], 
                    100.0*plot_data[(experiment_pattern, 'acc_tst_ci')][1], 
                    alpha=0.5, edgecolor=colors[i], facecolor=colors[i])

    legend = ax1.legend(shadow=True, loc=4)
    ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax1.grid(which='minor', alpha=0.5)                                                
    ax1.grid(which='major', alpha=1.0) 
    ax1.set_ylim([65,80])

    ax1.set_title(ROOT_DIR)
    ax1.set_xlabel('train iterations')
    ax1.set_xticks(range(0, 50001, 5000))
    ax1.set_ylabel('Accuracy target test')
    ax1.yaxis.set_major_locator(majorLocator)
    ax1.yaxis.set_major_formatter(majorFormatter)
    plt.xticks(rotation=45)
    
    

for i, experiment_pattern in enumerate(experiement_patterns):
    

    ax2.plot(100.0*plot_data[(experiment_pattern, 'acc_val_mean')], label=experiment_pattern, c=colors[i])
    ax2.fill_between(plot_data[(experiment_pattern, 'acc_val_mean')].index, 
                    100.0*plot_data[(experiment_pattern, 'acc_val_ci')][0], 
                    100.0*plot_data[(experiment_pattern, 'acc_val_ci')][1], 
                    alpha=0.5, edgecolor=colors[i], facecolor=colors[i])

    legend = ax2.legend(shadow=True, loc=4)
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax2.grid(which='minor', alpha=0.5)                                                
    ax2.grid(which='major', alpha=1.0) 
    ax2.set_ylim([65,80])

    ax2.set_title(ROOT_DIR)
    ax2.set_xlabel('train iterations')
    ax2.set_xticks(range(0, 50001, 5000))
    ax2.set_ylabel('Accuracy target validation')
    ax2.yaxis.set_major_locator(majorLocator)
    ax2.yaxis.set_major_formatter(majorFormatter)
    plt.xticks(rotation=45)
        

plt.grid('on')
plt.show()

In [ ]:
acc_tst.loc[acc_tst.idxmax(axis=1).dropna().values==acc_tst.columns[0], acc_tst.columns[0]]


def get_best_over_val(acc_tst, acc_val):
    df = pd.DataFrame(index=acc_tst.index, columns=['acc_tst'])
    for column in acc_val.columns:
        idx=acc_val.idxmax(axis=1).dropna().values==column
        df.loc[idx,'acc_tst'] = acc_tst.loc[idx, column].values
    return df


fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(2,1,1)
ax1.set_ylim([60,75])
plt.grid('on')
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    acc_tst_best = get_best_over_val(acc_tst, acc_val)
    ax1.plot(100*acc_tst_best, label=experiment_pattern, c=colors[i])
    legend = ax1.legend(shadow=True, loc=4)
    
    ax1.set_title(ROOT_DIR)
    ax1.set_xlabel('train iterations')
    ax1.set_xticks(range(0, 50001, 10000))
    ax1.set_ylabel('Accuracy target test, best over validation set')
    ax1.yaxis.set_major_locator(majorLocator)
    ax1.yaxis.set_major_formatter(majorFormatter)
    plt.xticks(rotation=45)

In [ ]:
def argmax_to_max(arr, argmax, axis):
    """argmax_to_max(arr, arr.argmax(axis), axis) == arr.max(axis)"""
    new_shape = list(arr.shape)
    del new_shape[axis]

    grid = np.ogrid[tuple(map(slice, new_shape))]
    grid.insert(axis, argmax)

    return arr[tuple(grid)]


def get_max_over_val_bootstrapped(acc_tst, acc_val, bootstrap_ratio, n_mc_trials, replace):
    acc_tst = acc_tst.values
    acc_val = acc_val.values
    assert acc_val.shape[1] == acc_tst.shape[1]
    output = np.zeros(shape=(acc_tst.shape[0], n_mc_trials))
    for i in range(n_mc_trials):
        col_dxs = np.random.choice(range(acc_val.shape[1]), size=round(bootstrap_ratio*acc_val.shape[1]), 
                                  replace=replace)
        acc_val_boot = acc_val[:,col_dxs]
        acc_tst_boot = acc_tst[:,col_dxs]
                
        val_max_idx = np.argmax(acc_val_boot, axis=1)
        output[:, i] = argmax_to_max(acc_tst_boot, val_max_idx, axis=1)

    return output

In [ ]:
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(2,1,1)
ax1.set_ylim([60,75])
ax2 = fig.add_subplot(2,1,2)
ax2.set_ylim([65,80])
plt.grid('on')
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    a = get_max_over_val_bootstrapped(acc_tst, acc_val, bootstrap_ratio=1.0, 
                                      n_mc_trials=1000, replace=True
                                     )
    
    a_ci = np.percentile(a, [5.0, 95.0], axis=1)
    
    ax1.plot(acc_tst.index, 100*np.mean(a, axis=1), label=experiment_pattern, c=colors[i])
    
    ax1.fill_between(acc_tst.index, 100.0*a_ci[0], 100.0*a_ci[1], 
                    alpha=0.5, edgecolor=colors[i], facecolor=colors[i])
    
    legend = ax1.legend(shadow=True, loc=4)
    ax1.grid(which='minor', alpha=0.5)                                                
    ax1.grid(which='major', alpha=1.0) 
    ax1.set_ylim([65,80])
    
    ax1.set_title(ROOT_DIR)
    ax1.set_xlabel('train iterations')
    ax1.set_xticks(range(0, 50001, 5000))
    ax1.set_ylabel('Accuracy target test, best over validation set')
    ax1.yaxis.set_major_locator(majorLocator)
    ax1.yaxis.set_major_formatter(majorFormatter)
    plt.xticks(rotation=45)
    
    
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_val')], plot_data[(experiment_pattern, 'acc_val')]
    a = get_max_over_val_bootstrapped(acc_tst, acc_val, bootstrap_ratio=1.0, 
                                      n_mc_trials=10000, replace=True
                                     )
    
    a_ci = np.percentile(a, [5.0, 95.0], axis=1)
    
    ax2.plot(acc_tst.index, 100*np.mean(a, axis=1), label=experiment_pattern, c=colors[i])
    
    ax2.fill_between(acc_tst.index, 100.0*a_ci[0], 100.0*a_ci[1], 
                    alpha=0.5, edgecolor=colors[i], facecolor=colors[i])
    
    legend = ax2.legend(shadow=True, loc=4)
    ax2.grid(which='minor', alpha=0.5)                                                
    ax2.grid(which='major', alpha=1.0) 
    ax2.set_ylim([65,80])
    
    ax2.set_title(ROOT_DIR)
    ax2.set_xlabel('train iterations')
    ax2.set_xticks(range(0, 50001, 5000))
    ax2.set_ylabel('Accuracy target validation, best over validation set')
    ax2.yaxis.set_major_locator(majorLocator)
    ax2.yaxis.set_major_formatter(majorFormatter)
    plt.xticks(rotation=45)
    
plt.grid('on')
plt.show()

In [ ]:
def get_early_stop_over_val(acc_tst, acc_val):
    acc_tst = np.nan_to_num(acc_tst.values)
    acc_val = np.nan_to_num(acc_val.values)
    assert acc_val.shape[1] == acc_tst.shape[1]
        
    val_max_idx = np.argmax(acc_val, axis=0)
    acc_tst_max = argmax_to_max(acc_tst, val_max_idx, axis=0)
    
    return acc_tst_max

print("Experiment root folder: %s" %ROOT_DIR)
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    acc_tst_validated = get_early_stop_over_val(acc_tst, acc_val)
    
    acc_tst_validated_ci = sms.DescrStatsW(acc_tst_validated.T).tconfint_mean()
    acc_tst_validated_mean = acc_tst_validated.mean()
    
    print('Experiment name: %s' %(experiment_pattern))
    print('mean accuracy: %.1f +/- %.1f' %(100.0*acc_tst_validated_mean, 
            100.0*(acc_tst_validated_ci[1]-acc_tst_validated_mean)))

In [ ]:
print("Experiment root folder: %s" %ROOT_DIR)
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    acc_tst_validated = get_early_stop_over_val(acc_val, acc_val)
    
    acc_tst_validated_ci = sms.DescrStatsW(acc_tst_validated.T).tconfint_mean()
    acc_tst_validated_mean = acc_tst_validated.mean()
    
    print('Experiment name: %s' %(experiment_pattern))
    print('mean accuracy, validation: %.1f +/- %.1f' %(100.0*acc_tst_validated_mean, 
            100.0*(acc_tst_validated_ci[1]-acc_tst_validated_mean)))